In [12]:
import numpy as np
import pandas as pd 
from pathlib import Path

path = Path("./us-patent-phrase-to-phrase-matching")
df = pd.read_csv(path/"train.csv")

In [13]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [18]:
df.describe(include="object")

,id,anchor,target,context,input
count,36473,36473,36473,36473,36473
unique,36473,733,29340,106,36473
top,37d61fd2272659b1,component composite coating,composition,H01,TEXT 1: A47; TEXT 2: abatement of pollution; A...
freq,1,152,24,2186,1


In [21]:
df['input'] = "TEXT 1: " + df.context + "; TEXT 2: " + df.target + "; ANC: " + df.anchor

In [22]:
df.input.head

<bound method NDFrame.head of 0        TEXT 1: A47; TEXT 2: abatement of pollution; A...
1        TEXT 1: A47; TEXT 2: act of abating; ANC: abat...
2        TEXT 1: A47; TEXT 2: active catalyst; ANC: aba...
3        TEXT 1: A47; TEXT 2: eliminating process; ANC:...
4        TEXT 1: A47; TEXT 2: forest region; ANC: abate...
                               ...                        
36468    TEXT 1: B44; TEXT 2: wooden article; ANC: wood...
36469    TEXT 1: B44; TEXT 2: wooden box; ANC: wood art...
36470    TEXT 1: B44; TEXT 2: wooden handle; ANC: wood ...
36471    TEXT 1: B44; TEXT 2: wooden material; ANC: woo...
36472    TEXT 1: B44; TEXT 2: wooden substrate; ANC: wo...
Name: input, Length: 36473, dtype: object>

In [23]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)

In [24]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [2]:
model_nm = 'microsoft/deberta-v3-small'

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/homebrew/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokz.tokenize("Hello, Mr. Mogi")

['▁Hello', ',', '▁Mr', '.', '▁Mog', 'i']

In [6]:
def tok_func(x): return tokz(x["input"])

In [25]:
tok_ds = ds.map(tok_func, batched=True)

In [26]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT 1: A47; TEXT 2: abatement of pollution; ANC: abatement',
 [1,
  54453,
  376,
  294,
  336,
  5753,
  346,
  54453,
  392,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  294,
  47284,
  2])

In [32]:
tokz.vocab['of']

1580

In [35]:
tok_ds = tok_ds.rename_columns({'score': 'labels'})

In [37]:
eval_df = pd.read_csv(path/'test.csv')